In [1]:
from numpy import *
from scipy.sparse import *
from sklearn.preprocessing import *

In [2]:
R=load_npz("all_playlist_with_tracks_URM.npz")
R

<45649x100000 sparse matrix of type '<class 'numpy.int32'>'
	with 1040522 stored elements in Compressed Sparse Column format>

In [3]:
trackArtist=load_npz("fucking_ordered_track_artist.npz")
trackAlbum=load_npz("fucking_ordered_track_album.npz")

In [4]:
simByArtist=trackArtist*trackArtist.T
simByAlbum=trackAlbum*trackAlbum.T
simByPlaylist=R.T*R

In [5]:
simByAlbum.setdiag(0)
simByArtist.setdiag(0)
simByPlaylist.setdiag(0)

/home/oppy/.local/lib/python3.6/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [6]:
trackIDs=load("fucking_ordered_tracks.npy")
targetTrack=unique(genfromtxt("target_tracks.csv",dtype=int32,skip_header=1))
maskT=list(map(lambda x:x in targetTrack,trackIDs))

In [7]:
maskT=array(maskT)
maskT

array([ True, False, False, ...,  True,  True,  True], dtype=bool)

In [10]:
simByPlaylist=normalize(simByPlaylist)

In [11]:
simByArtist,simByAlbum, simByPlaylist

(<100000x100000 sparse matrix of type '<class 'numpy.int64'>'
 	with 2590190 stored elements in Compressed Sparse Row format>,
 <100000x100000 sparse matrix of type '<class 'numpy.int64'>'
 	with 420906 stored elements in Compressed Sparse Row format>,
 <100000x100000 sparse matrix of type '<class 'numpy.float64'>'
 	with 60319472 stored elements in Compressed Sparse Row format>)

In [12]:
S=simByArtist+1.5*simByAlbum+simByPlaylist

In [13]:
S=S[:,maskT]

In [14]:
RR=R*S
RR

<45649x32195 sparse matrix of type '<class 'numpy.float64'>'
	with 319129815 stored elements in Compressed Sparse Column format>

In [15]:
allPlaylist=load("fucking_ordered_playlist.npy")

In [16]:
allPlaylist

array([    7569,     7614,     7641, ..., 11765405, 11766293, 11766362], dtype=int32)

In [17]:
allTracks=trackIDs
allTracks,allTracks.size

(array([    252,     360,     394, ..., 5007354, 5014702, 5018274], dtype=int32),
 100000)

In [27]:
R=R[:,maskT]

In [44]:
RR,R

(<45649x32195 sparse matrix of type '<class 'numpy.float64'>'
 	with 319129815 stored elements in Compressed Sparse Column format>,
 <45649x32195 sparse matrix of type '<class 'numpy.int32'>'
 	with 564194 stored elements in Compressed Sparse Column format>)

In [45]:
R.nonzero()

(array([    0,     0,     0, ..., 45648, 45648, 45648], dtype=int32),
 array([ 1085,  6346, 23765, ..., 22118, 23274, 26068], dtype=int32))

In [46]:
RR[R.nonzero()]=0
RR

/home/oppy/.local/lib/python3.6/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


MemoryError: 

In [14]:
targetTrack=unique(genfromtxt("target_tracks.csv",dtype=int32,skip_header=1))
targetTrack.size

32195

In [15]:
maskT=list(map(lambda x:x in targetTrack,allTracks))
len(maskT)

100000

In [16]:
targetPlaylist=unique(genfromtxt("target_playlists.csv",skip_header=1,dtype=int32))
targetPlaylist.size

10000

In [17]:
maskP=list(map(lambda x:x in targetPlaylist,allPlaylist))
len(maskP)

45649

In [ ]:
with open("noTagsArtist1Album1,5.csv","w") as f:
    f.write("playlist_id,track_ids\n")
    for i,sims in enumerate(RR):
        if(maskP[i]):
            f.write(str(allPlaylist[i])+",")
            suggestions=allTracks[flip(argsort(sims.multiply(maskT).todense()),1).take(range(5))]
            for t in suggestions[0]:
                f.write(str(t)+" ")
            f.write("\n")